In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.decomposition import PCA
from PIL import Image
import glob
import warnings
import shap
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import random

warnings.filterwarnings('ignore')

## How to make labels of tracking datasets

I tried to make labels of tracking data because I think these have some features.

I changed the tracking datasets to png datasets.
And I made labels of these png datasets eith PCA and KMeans.

I made the png datasets of tracking data only 2020.
Because using all datasets needs too big memories.

If you think that this is useful ,please upvote.
Your upvote makes me so encouraged.

In [ ]:
filepath1='../input/nfl-make-png-2020-1-w/'
list_2020_1=os.listdir(filepath1)

filepath2='../input/nfl-make-png-2020-2-w/'
list_2020_2=os.listdir(filepath2)

filepath3='../input/nfl-make-png-2020-3-w/'
list_2020_3=os.listdir(filepath3)

In [ ]:
list_2020_1_=[i for i in list_2020_1 if 'png' in i]
list_2020_2_=[i for i in list_2020_2 if 'png' in i]
list_2020_3_=[i for i in list_2020_3 if 'png' in i]

In [ ]:
#load png data
image_size = 64

x = []
png_data=[]
for i in list_2020_1_:
    image = Image.open(filepath1+i)
    #image = image.convert('L')  
    image = image.resize((image_size, image_size)) 
    data = np.asarray(image)
    x.append(data)
    png_data.append(i)
    
for i in list_2020_2_:
    image = Image.open(filepath2+i)
    #image = image.convert('L')  
    image = image.resize((image_size, image_size)) 
    data = np.asarray(image)
    x.append(data)
    png_data.append(i)
    
for i in list_2020_3_:
    image = Image.open(filepath3+i)
    #image = image.convert('L')  
    image = image.resize((image_size, image_size)) 
    data = np.asarray(image)
    x.append(data)
    png_data.append(i)
    
X = np.array(x)
X = X.reshape(X.shape[0], image_size * image_size,4) 
X = X / 255.0  


print('X.shape =', X.shape)
rows, cols = 5, 10  
fig, aX_invs = plt.subplots(ncols=cols, nrows=rows, figsize=(18, 10)) 
for i in range(50):
    r = i // cols
    c = i % cols
    aX_invs[r, c].imshow(X[i].reshape(image_size,image_size,4),vmin=0.0,vmax=1.0, cmap = cm.Greys_r)
    aX_invs[r, c].set_title('data %d' % (i+1))
    aX_invs[r, c].get_xaxis().set_visible(False)
    aX_invs[r, c].get_yaxis().set_visible(False)
plt.show()

In [ ]:
X2=X.reshape(X.shape[0],4096*4)
X2.shape

# PCA

In [ ]:
# PCA(Principal component analysis)
N = 400  #400 component
pca = PCA(n_components=N)
pca.fit(X2)
print('n_components = '+str(N))
print('explained_variance_ratio = ', pca.explained_variance_ratio_.sum())


In [ ]:
#Dimensionality reduction and dimension restoration coefficients
X_trans = pca.transform(X2)
X_inv = pca.inverse_transform(X_trans)
print ('X.shape =', X.shape)
print ('X_trans.shape =', X_trans.shape)
print ('X_inv.shape =', X_inv.shape)


rows, cols = 2, 8  # 2行8列
fig, aX_invs = plt.subplots(ncols=cols, nrows=rows, figsize=(18,4))
for i in range(8):
    aX_invs[0, i].imshow(X[i,:].reshape(image_size, image_size,4),vmin=0.0,vmax=1.0, cmap = cm.Greys_r)
    aX_invs[0, i].set_title('original %d' % (i+1))
    aX_invs[0, i].get_xaxis().set_visible(False)
    aX_invs[0, i].get_yaxis().set_visible(False)
    aX_invs[1, i].imshow(X_inv[i,:].reshape(image_size, image_size,4),vmin=0.0,vmax=1.0, cmap = cm.Greys_r)
    aX_invs[1, i].set_title('restore %d' % (i+1))
    aX_invs[1, i].get_xaxis().set_visible(False)
    aX_invs[1, i].get_yaxis().set_visible(False)

# KMeans

In [ ]:
import numpy as np
import itertools 

#Kmeans
class KMeans:
    def __init__(self, n_clusters, max_iter = 1000, random_seed = 0):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.random_state = np.random.RandomState(random_seed)

    def fit(self, X):
        cycle = itertools.cycle(range(self.n_clusters))
        self.labels_ = np.fromiter(itertools.islice(cycle, X.shape[0]), dtype = np.int)
        self.random_state.shuffle(self.labels_)
        labels_prev = np.zeros(X.shape[0])
        count = 0
        self.cluster_centers_ = np.zeros((self.n_clusters, X.shape[1]))

        
        while (not (self.labels_ == labels_prev).all() and count < self.max_iter):
            for i in range(self.n_clusters):
                XX = X[self.labels_ == i, :]
                self.cluster_centers_[i, :] = XX.mean(axis = 0)
            dist = ((X[:, :, np.newaxis] - self.cluster_centers_.T[np.newaxis, :, :]) ** 2).sum(axis = 1)
            labels_prev = self.labels_
            self.labels_ = dist.argmin(axis = 1)
            count += 1

    def predict(self, X):
        dist = ((X[:, :, np.newaxis] - self.cluster_centers_.T[np.newaxis, :, :]) ** 2).sum(axis = 1)
        labels = dist.argmin(axis = 1)
        return labels

In [ ]:

#make 8 labels
model =  KMeans(8)
model.fit(X_trans)

print(model.labels_)

In [ ]:
p_0=X_inv[model.labels_ == 0, :]#label0 no gazou(*4096)
p_1=X_inv[model.labels_ == 1, :]
p_2=X_inv[model.labels_ == 2, :]
p_3=X_inv[model.labels_ == 3, :]
p_4=X_inv[model.labels_ == 4, :]
p_5=X_inv[model.labels_ == 5, :]
p_6=X_inv[model.labels_ == 6, :]
p_7=X_inv[model.labels_ == 7, :]

In [ ]:
len(model.labels_)

confirm how png datasets are labeled.

In [ ]:
rows, cols = 8, 8
fig, aX_invs = plt.subplots(ncols=cols, nrows=rows, figsize=(18,25))
for i in range(8):
    aX_invs[0, i].imshow(p_0[i,:].reshape(image_size, image_size,4),vmin=0.0,vmax=1.0, cmap = cm.Greys_r)
    aX_invs[0, i].set_title('label0 %d' % (i+1))
    aX_invs[0, i].get_xaxis().set_visible(False)
    aX_invs[0, i].get_yaxis().set_visible(False)
    aX_invs[1, i].imshow(p_1[i,:].reshape(image_size, image_size,4),vmin=0.0,vmax=1.0, cmap = cm.Greys_r)
    aX_invs[1, i].set_title('label1 %d' % (i+1))
    aX_invs[1, i].get_xaxis().set_visible(False)
    aX_invs[1, i].get_yaxis().set_visible(False)
    aX_invs[2, i].imshow(p_2[i,:].reshape(image_size, image_size,4),vmin=0.0,vmax=1.0, cmap = cm.Greys_r)
    aX_invs[2, i].set_title('label2 %d' % (i+1))
    aX_invs[2, i].get_xaxis().set_visible(False)
    aX_invs[2, i].get_yaxis().set_visible(False)
    aX_invs[3, i].imshow(p_3[i,:].reshape(image_size, image_size,4),vmin=0.0,vmax=1.0, cmap = cm.Greys_r)
    aX_invs[3, i].set_title('label3 %d' % (i+1))
    aX_invs[3, i].get_xaxis().set_visible(False)
    aX_invs[3, i].get_yaxis().set_visible(False)
    aX_invs[4, i].imshow(p_4[i,:].reshape(image_size, image_size,4),vmin=0.0,vmax=1.0, cmap = cm.Greys_r)
    aX_invs[4, i].set_title('label4 %d' % (i+1))
    aX_invs[4, i].get_xaxis().set_visible(False)
    aX_invs[4, i].get_yaxis().set_visible(False)
    aX_invs[5, i].imshow(p_5[i,:].reshape(image_size, image_size,4),vmin=0.0,vmax=1.0, cmap = cm.Greys_r)
    aX_invs[5, i].set_title('label5 %d' % (i+1))
    aX_invs[5, i].get_xaxis().set_visible(False)
    aX_invs[5, i].get_yaxis().set_visible(False)
    aX_invs[6, i].imshow(p_6[i,:].reshape(image_size, image_size,4),vmin=0.0,vmax=1.0, cmap = cm.Greys_r)
    aX_invs[6, i].set_title('label6 %d' % (i+1))
    aX_invs[6, i].get_xaxis().set_visible(False)
    aX_invs[6, i].get_yaxis().set_visible(False)
    aX_invs[7, i].imshow(p_7[i,:].reshape(image_size, image_size,4),vmin=0.0,vmax=1.0, cmap = cm.Greys_r)
    aX_invs[7, i].set_title('label7 %d' % (i+1))
    aX_invs[7, i].get_xaxis().set_visible(False)
    aX_invs[7, i].get_yaxis().set_visible(False)



It's almost working!

### concat labels that I made and other data
.

In [ ]:
label_data=pd.DataFrame(np.arange(len(X)),columns={'number'})
label_data['labels']=model.labels_
label_data['png_data']=png_data
label_data

In [ ]:
#df=pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2020.csv')
#df

In [ ]:
a_a=label_data['png_data'].str.split('.', expand=True)
a_a.rename(columns={0: 'ID', 1: 'png'}, inplace=True)
label_data['ID']=a_a['ID']

In [ ]:
play=pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')
play['gameId']=play['gameId'].astype(str)
play['playId']=play['playId'].astype(str)
play['ID']=play['gameId'].str.cat(play['playId'], sep='_')

In [ ]:
pd.set_option('display.max_columns', 50)
data_concat=pd.merge(play,label_data, on='ID')
data_concat

## playResult vs labels of tracking data separate with specialTeamsPlayType ****

In [ ]:
data_concat['specialTeamsPlayType'].unique()

In [ ]:
plt.style.use('fivethirtyeight')
plt.figure(figsize=(16,10))
plt.grid(False)
data_ko=data_concat[data_concat['specialTeamsPlayType']=='Kickoff']
data_ep=data_concat[data_concat['specialTeamsPlayType']=='Extra Point']
data_punt=data_concat[data_concat['specialTeamsPlayType']=='Punt']
data_fg=data_concat[data_concat['specialTeamsPlayType']=='Field Goal']
plt.scatter(x=data_ko['labels'],y=data_ko['playResult'],c='r',label='Kickoff',alpha=0.3)
plt.scatter(x=data_ep['labels'],y=data_ep['playResult'],c='g',label='Extra Point',alpha=0.3)
plt.scatter(x=data_punt['labels'],y=data_punt['playResult'],c='y',label='Punt',alpha=0.3)
plt.scatter(x=data_fg['labels'],y=data_fg['playResult'],c='b',label='Field Goal',alpha=0.3)
plt.title("playResult vs labels of tracking data separate with specialTeamsPlayType")
plt.xlabel("labels of tracking data")
plt.ylabel("playResult")
plt.legend()

In [ ]:
data_concat

# ****LGBM and SHAP with labels of tracking datasets****

In [ ]:
#Code by rossinEndrew https://www.kaggle.com/endrewrossin/fast-initial-lightgbm-model-to-detect-exam-result/comments

SEED = 99
random.seed(SEED)
np.random.seed(SEED)
#Code by rossinEndrew https://www.kaggle.com/endrewrossin/fast-initial-lightgbm-model-to-detect-exam-result/comments
data_concat=data_concat.drop(['playDescription','gameClock','playId','png_data','number','ID','gameId'], axis = 1)
playsmodel = data_concat.copy()

# read the "object" columns and use labelEncoder to transform to numeric
for col in playsmodel.columns[playsmodel.dtypes == 'object']:
    le = LabelEncoder()
    playsmodel[col] = playsmodel[col].astype(str)
    le.fit(playsmodel[col])
    playsmodel[col] = le.transform(playsmodel[col])

In [ ]:
playsmodel

In [ ]:
X = playsmodel.drop(['playResult'], axis = 1)
y = playsmodel['playResult']
lgb_params = {
                    'objective':'binary',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.005,
                    'num_leaves': 20,
                    'max_depth':-1,
                    'subsample':0.9,
                    'n_estimators':2500,
                    'seed': SEED,
                    'early_stopping_rounds':100, 
                }

In [ ]:
#Code by rossinEndrew https://www.kaggle.com/endrewrossin/fast-initial-lightgbm-model-to-detect-exam-result/comments

# choose the number of folds, and create a variable to store the auc values and the iteration values.
K = 5
folds = KFold(K, shuffle = True, random_state = SEED)
best_scorecv= 0
best_iteration=0

# Separate data in folds, create train and validation dataframes, train the model and cauculate the mean AUC.
for fold , (train_index,test_index) in enumerate(folds.split(X, y)):
    print('Fold:',fold+1)
          
    X_traincv, X_testcv = X.iloc[train_index], X.iloc[test_index]
    y_traincv, y_testcv = y.iloc[train_index], y.iloc[test_index]
    
    train_data = lgb.Dataset(X_traincv, y_traincv)
    val_data   = lgb.Dataset(X_testcv, y_testcv)
    
    LGBM = lgb.train(lgb_params, train_data, valid_sets=[train_data,val_data], verbose_eval=250)
    best_scorecv += LGBM.best_score['valid_1']['auc']
    best_iteration += LGBM.best_iteration

best_scorecv /= K
best_iteration /= K
print('\n Mean AUC score:', best_scorecv)
print('\n Mean best iteration:', best_iteration)

In [ ]:
#XLGB SHAP
#Code by rossinEndrew https://www.kaggle.com/endrewrossin/fast-initial-lightgbm-model-to-detect-exam-result/comments

lgb_params = {
                    'objective':'binary',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.05,
                    'num_leaves': 20,
                    'max_depth':-1,
                    'subsample':0.9,
                    'n_estimators':round(best_iteration),
                    'seed': SEED,
                    'early_stopping_rounds':None, 
                }

train_data_final = lgb.Dataset(X, y)
LGBM = lgb.train(lgb_params, train_data)

In [ ]:
# telling wich model to use
explainer = shap.TreeExplainer(LGBM)
# Calculating the Shap values of X features
shap_values = explainer.shap_values(X)

In [ ]:
shap.summary_plot(shap_values[0], X, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values[0], X)

# Conclusion

'playResult' have relations with the 'specialTeamPlayType' the most.
Labels that I made in this notebook had a little impact.
But they are similar to SpecialTeamPlayType.

If you apply this method,we can label the movements of one player and a ball.
It may lead to better results...

Thank you for watching this notebook till the end.

